In [1]:
from rcd_dataSet import RCD_DataSet


In [4]:
# ----------基本参数--------------
datadir = '../../'
concept_graph_dir = './graph/'
dataSet_list = ('FrcSub', 'Math1', 'Math2', 'ASSIST_0910', 'ASSIST_2017', 'MAT_2016', 'JUNYI')

dataSet_idx = 3
test_ratio = 0.8
batch_size = 64
learn_rate = 3e-2
emb_dim = 16

data_set_name = dataSet_list[dataSet_idx]
epochs = 8
device = 'cuda'
# ----------基本参数--------------

dataSet = RCD_DataSet(datadir, concept_graph_dir, data_set_name)

DataSet: ASSIST_0910


In [5]:
dataSet.all_graph

HeteroData(
  stu={ num_nodes=2392 },
  item={ num_nodes=17657 },
  conc={ num_nodes=116 },
  (stu, answer, item)={ edge_index=[2, 265900] },
  (item, contain, conc)={ edge_index=[2, 17657] },
  (conc, relevant, conc)={ edge_index=[2, 3364] }
)

In [12]:
len(dataSet.item_conc_df['concept'].unique())

97

In [19]:
import pandas as pd
def get_item_concept_df(item) -> pd.DataFrame:
    item = item[~item.index.duplicated()]  # 去除重复项
    item_list, concept_list = [], []
    for idx in item.index:
        now_concept_list = eval(item.loc[idx, 'knowledge_code'])
        item_list.extend([idx] * len(now_concept_list))
        concept_list.extend(now_concept_list)
    return pd.DataFrame({'item': item_list, 'concept': concept_list}).astype('int')

In [20]:
tmp=get_item_concept_df(dataSet.item)

In [21]:
len(tmp['concept'].unique())

97

In [ ]:
def get_item_concept_df2(item) -> pd.DataFrame:
    item = item[~item.index.duplicated()]  # 去除重复项
    item_list, concept_list = [], []
    for idx in item.index:
        now_concept_list = eval(item.loc[idx, 'knowledge_code'])
        item_list.extend([idx] * len(now_concept_list))
        concept_list.extend(now_concept_list)
    return pd.DataFrame({'item': item_list, 'concept': concept_list}).astype('int')

In [22]:
dataSet.item

,knowledge_code
item_id,
1,[10]
2,[1]
3,[10]
4,[1]
5,[10]
...,...
17653,[94]
17654,[94]
17655,[94]


In [19]:
train, test = dataSet.get_train_test(dataSet.record, test_ratio=0.2)

test_ratio: 0.2


In [20]:
data_list=dataSet.get_data_loader(train,batch_size=batch_size)

[Graphing:]: 100%|██████████| 9/9 [00:00<00:00, 29.39it/s]


In [6]:
len(train)

8576

In [21]:
data_list[0]

HeteroData(
  triple_index=[3, 2855],
  mean_index=[2855],
  label=[1024],
  stu={
    num_nodes=64,
    pos=[64]
  },
  item={
    num_nodes=20,
    pos=[20]
  },
  conc={
    num_nodes=8,
    pos=[8]
  },
  (stu, answer, item)={ edge_index=[2, 1024] },
  (item, contain, conc)={ edge_index=[2, 56] },
  (stu, master, conc)={ edge_index=[2, 512] }
)

In [22]:
from models import RCD

model=RCD(dataSet.stu_num,dataSet.prob_num,dataSet.concept_num,emb_dim=emb_dim,learn_rate=learn_rate)

Total number of paramerters in networks is 12721  


In [23]:
model.fit(dataSet.all_graph,data_list,epochs=2)

[Epoch:1]: 100%|██████████| 9/9 [00:00<00:00, 20.99it/s]


	1 epoch loss = 0.7065359817610847


[Epoch:2]: 100%|██████████| 9/9 [00:00<00:00, 26.70it/s]

	2 epoch loss = 0.6982975900173187


HeteroData(
  stu={
    num_nodes=536,
    x=[536, 16]
  },
  item={
    num_nodes=20,
    x=[20, 16]
  },
  conc={
    num_nodes=8,
    x=[8, 16]
  },
  (stu, answer, item)={ edge_index=[2, 10720] },
  (item, contain, conc)={ edge_index=[2, 56] },
  (conc, relevant, conc)={ edge_index=[2, 53] }
)

In [1]:
from rcd_dataSet import RCD_DataSet

datadir = '../../'
concept_graph_dir = './graph/'
dataSet_list = ('FrcSub','ASSIST_0910', 'ASSIST_2017', 'JUNYI', 'MathEC', 'KDDCUP')

dataSet_idx = 0
test_ratio = 0.2
batch_size = 32
learn_rate = 3e-3

data_set_name = dataSet_list[dataSet_idx]
epochs = 80
# ----------基本参数--------------

dataSet = RCD_DataSet(datadir, concept_graph_dir, data_set_name)

train, test = dataSet.get_train_test(dataSet.record, test_ratio=test_ratio)
# data_list = dataSet.get_data_loader(train, batch_size=batch_size)
test_data_list = dataSet.get_data_loader(test, batch_size=batch_size)

DataSet: FrcSub
test_ratio: 0.2


[Graphing:]: 100%|██████████| 536/536 [00:02<00:00, 247.02it/s]


In [4]:
for x in test_data_list:
    break

In [5]:
x

RCDDataBatch(
  stu_track=[355],
  item_track=[355],
  conc_track=[355],
  mean_track=[355],
  label=[128],
  stu={
    num_nodes=32,
    pos=[32],
    batch=[32],
    ptr=[33]
  },
  item={
    num_nodes=128,
    pos=[128],
    batch=[128],
    ptr=[33]
  },
  conc={
    num_nodes=184,
    pos=[184],
    batch=[184],
    ptr=[33]
  },
  (stu, answer, item)={ edge_index=[2, 128] },
  (item, contain, conc)={ edge_index=[2, 355] },
  (stu, master, conc)={ edge_index=[2, 184] }
)

In [9]:
x.conc_track

tensor([  0,   1,   2,   3,   3,   4,   3,   5,   3,   6,   7,   8,   9,  10,
          7,   8,   9,   6,  10,   8,  11,   8,  12,  13,  14,  15,  13,  14,
         14,  15,  14,  16,  17,  18,  19,  20,  16,  20,  17,  20,  17,  21,
         22,  23,  24,  25,  22,  26,  25,  27,  25,  27,  21,  25,  28,  29,
         30,  31,  32,  28,  33,  29,  31,  28,  34,  29,  31,  28,  34,  31,
         35,  36,  37,  38,  39,  36,  40,  38,  41,  38,  38,  42,  43,  44,
         45,  46,  45,  46,  47,  45,  47,  43,  48,  49,  45,  50,  51,  52,
         53,  52,  50,  52,  54,  50,  55,  51,  52,  56,  57,  56,  57,  58,
         59,  56,  57,  60,  59,  56,  61,  62,  63,  64,  64,  65,  64,  61,
         63,  66,  64,  67,  68,  69,  67,  70,  69,  71,  67,  69,  70,  69,
         72,  72,  73,  74,  72,  74,  75,  76,  72,  77,  78,  79,  80,  81,
         79,  82,  79,  80,  83,  83,  84,  85,  86,  87,  85,  83,  88,  84,
         85,  89,  90,  91,  92,  89,  90,  93,  91,  89,  94,  

In [19]:
dataSet.all_graph

RCDData(
  stu={ num_nodes=536 },
  item={ num_nodes=20 },
  conc={ num_nodes=8 },
  (stu, answer, item)={ edge_index=[2, 10720] },
  (item, contain, conc)={ edge_index=[2, 56] },
  (conc, relevant, conc)={ edge_index=[2, 53] }
)